# Cut physionet dataset so that we have different time intervals

In [1]:
import os
import sys
os.chdir("/Users/luisa/Desktop/nygc/cluster/projects/fl4tsf/bin")
sys.path.append(os.getcwd()) 
import torch

from flower.task import Net, train
import torch
from torch.utils.data import DataLoader
from lib.dataset_utils import *
%matplotlib inline
%load_ext autoreload
%autoreload 2
from lib.physionet import PhysioNet, variable_time_collate_fn, get_data_min_max
from sklearn import model_selection
from torch.utils.data import DataLoader
from lib import utils
from lib.dataset_utils import store_dataset_physionet

path_prefix = "/Users/luisa/Desktop/nygc/cluster/projects/fl4tsf/data"
prefix = "physionet"
batch_size = 64
classif = False
noise_weight = 0.1
max_t_extrap = 5
n_total_tp = 100
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
amp_start = 1.0
amp_end = 1.0 
freq_start =1.0
freq_end = freq_start
epochs = 5
batch_size = 3
sample_tp = 0.9
n_samples = 4
max_t_extrap = 50
lr_val = 0.01
cut_tp = None
extrap = None

from types import SimpleNamespace
args = SimpleNamespace()
args.sample_tp = 0
args.cut_tp = None
args.extrap = None

2025-07-01 17:06:08,994	INFO util.py:154 -- Outdated packages:
  ipywidgets==7.8.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [4]:
# Load physionet
data_train_0 = torch.load(f"/Users/luisa/Desktop/nygc/cluster/projects/fl4tsf/data/physionet_filtered/client_0_train.pt", weights_only=False)
data_test_0 = torch.load(f"/Users/luisa/Desktop/nygc/cluster/projects/fl4tsf/data/physionet_filtered/client_0_test.pt", weights_only=False)
data_train_1 = torch.load(f"/Users/luisa/Desktop/nygc/cluster/projects/fl4tsf/data/physionet_filtered/client_1_train.pt", weights_only=False)
data_test_1 = torch.load(f"/Users/luisa/Desktop/nygc/cluster/projects/fl4tsf/data/physionet_filtered/client_1_test.pt", weights_only=False)

In [22]:
# in client 0 we cut the amount of time to the first 20

def subset(data_train_0):
    new_data_train_0 = []
    for i in range(len(data_train_0)):
        id = data_train_0[i][0]
        x = data_train_0[i][1]
        y = data_train_0[i][2]
        mask = data_train_0[i][3]
        label = data_train_0[i][4]
        new_data_train_0.append((id, x[:20], y[:20], mask[:20], label))
    return new_data_train_0



In [24]:
# subset all data
new_data_train_0 = subset(data_train_0)
new_data_test_0 = subset(data_test_0)
new_data_train_1 = subset(data_train_1)
new_data_test_1 = subset(data_test_1)

total_dataset = new_data_train_0 + new_data_test_0 + new_data_train_1 + new_data_test_1

In [25]:
data_min, data_max = get_data_min_max(total_dataset)

Getting data min and max devices:  cpu


## Load dataset 

In [26]:
#################################################################
# Physionet Dataset - STORE
#################################################################
path_prefix = "/Users/luisa/Desktop/nygc/cluster/projects/fl4tsf/data"
prefix = "physionet_cut"


store_dataset_physionet(new_data_train_0, new_data_test_0, dataset_prefix=prefix, path_prefix=path_prefix, client_prefix=f"client_0")
store_dataset_physionet(new_data_train_1, new_data_test_1, dataset_prefix=prefix, path_prefix=path_prefix, client_prefix=f"client_1")
torch.save(data_min, f"{path_prefix}/{prefix}/client_0_data_min.pt")
torch.save(data_max, f"{path_prefix}/{prefix}/client_0_data_max.pt")
torch.save(data_min, f"{path_prefix}/{prefix}/client_1_data_min.pt")
torch.save(data_max, f"{path_prefix}/{prefix}/client_1_data_max.pt")